<a href="https://colab.research.google.com/github/takahashi-tsubaki/TensorFlow/blob/friend_Recommend/FriendRecommend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import functools

import numpy as np
import tensorflow as tf


In [ ]:
TRAIN_DATA_URL = "http://www.mrjyuryoku.com/model-Data.csv"
TEST_DATA_URL = "http://www.mrjyuryoku.com/test-Data.csv"

train_file_path = tf.keras.utils.get_file("model-Data.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("test-Data.csv", TEST_DATA_URL)

In [3]:
# numpy の値を読みやすくする
np.set_printoptions(precision=3, suppress=True)

データのロード
それではいつものように、扱っている CSV ファイルの先頭を見てみましょう。

In [ ]:
!head {train_file_path}

pandasでデータを読み込んでから Numpy の array を TensorFlow に渡すというのは可能です。もし、大規模なデータに対応するためにスケールアップしたり、データの読み込み処理を TensorFlow や tf.data に統合されたものにしたい場合、tf.data.experimental.make_csv_dataset 関数が利用できます。

In [5]:
LABELS = [0, 1, 2]
LABEL_COLUMN =  'CategoryID'

コンストラクタの引数の値が揃ったので、ファイルから CSV データを読み込みデータセットを作る。

（完全なドキュメントは、tf.data.experimental.make_csv_dataset を参照してください）

In [6]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [7]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

データセットを構成する要素は、(複数のサンプル, 複数のラベル)の形のタプルとして表されるバッチです。サンプル中のデータは（行ベースのテンソルではなく）列ベースのテンソルとして構成され、それぞれはバッチサイズ（このケースでは5個）の要素が含まれます。

実際に見てみましょう。

In [ ]:
show_batch(raw_train_data)

上で見たように、この CSV の列には名前がついています。Dataset のコンストラクターはこれらの列名を自動的に抽出します。一行目に列名が記されていない CSV を扱う場合には、列名のリストを make_csv_dataset 関数の column_names 引数に渡してください。

In [ ]:
CSV_COLUMNS = ['DataID','CategoryID', 'Category']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

このサンプルでは利用できるすべての列を使います。もし、データセットから特定の列を除外したい場合には、利用したい列名のリストを作成し、コンストラクタのオプショナルな引数である select_columns にそのリストを渡してください。

In [ ]:
SELECT_COLUMNS = ['DataID','CategoryID', 'Category']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

# データの前処理



CSV ファイルに含まれるデータには様々な型のものがありえます。データをモデルに入力する前に、典型的なケースでは、様々な型を含んだデータを固定長のベクトルに変換する必要があります。

TensorFlow には典型的なデータ変換を行う tf.feature_column が組み込まれています。詳細はこのチュートリアルを参照してください。

また、任意のツール (たとえば、nltk や sklearn) を使って前処理を行い、前処理した結果を TensorFlow にわたすこともできます。

モデルの内部で前処理を行う最大の利点は、モデルをエクスポートしたときにその中に前処理が含まれていることです。この方法を取れば、生データをそのままモデルに入力することができます。

すでにデータが適切に数値型になっている場合、モデルに渡す前にそのデータをベクトル形式にできます。

In [ ]:
SELECT_COLUMNS = ['DataID','CategoryID']
DEFAULTS = [0,0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

In [12]:
example_batch, labels_batch = next(iter(temp_dataset)) 

次のサンプルは、数値型の列をベクトルに変換するシンプルな関数の例です。

In [13]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

この関数をデータセットのそれぞれの要素に適用するする。

In [ ]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

様々な型を含んだデータセットを利用する場合、シンプルな数値型のフィールドだけを分離したくなるかもしれません。tf.feature_column はそのような処理を実現できますが、いくらかのオーバーヘッドを発生させるため、それが本当に必要になる場合以外ではできるだけ避けるべきでしょう。様々な型を含んだデータセットに話を戻しましょう。

In [ ]:
show_batch(raw_train_data)

In [16]:
example_batch, labels_batch = next(iter(temp_dataset))

数値型の特徴量を選んで、それらをベクトル化して単一の列に変換する。

In [17]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [18]:
NUMERIC_FEATURES = ['DataID']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [ ]:
show_batch(packed_train_data)

In [20]:
example_batch, labels_batch = next(iter(packed_train_data))

# データの正規化

In [ ]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

In [22]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [23]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

数値型の列を作成しましょう。tf.feature_columns.numeric_column では normalizer_fn 引数で正則化のための関数を受け取ります。また、渡された関数はそれぞれのバッチに対して実行されます。

functools.partial を用いて関数を部分適用し、normalize_numeric_data の引数 MEAN と STD を固定しておきましょう。

In [ ]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

モデルの訓練時には、数値データのブロックを選び出して利用できるように、作成した特徴量の列を入力に含めましょう。

In [ ]:
example_batch['numeric']

In [ ]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

カテゴリデータ
この CSV データ中のいくつかの列はカテゴリ列です。つまり、その中身は、限られた選択肢の中のひとつである必要があります。

tf.feature_column API を用いて、それぞれのカテゴリ列のためのコレクションを tf.feature_column.indicator_column で生成しましょう。

In [27]:
CATEGORIES = {
    'Category': ['cube', 'sphere','capsule'],
   
}

In [28]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [29]:
# See what you just created.
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Category', vocabulary_list=('cube', 'sphere', 'capsule'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [30]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[0. 0. 1.]


モデルを構築する際、これは入力レイヤーで行われるデータ処理の一部です。

# 一体化した前処理レイヤー

2つの特徴量列のコレクションを結合し、tf.keras.layers.DenseFeatures に入力して、両方のデータ型を読み込んで前処理する入力レイヤーを作成しましょう。

In [31]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [ ]:
print(preprocessing_layer(example_batch).numpy()[0])

# モデルの構築

preprocessing_layer から始まる tf.keras.Sequential を構築するする。

In [33]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

# 訓練、評価、そして予測

これでモデルをインスタンス化し、訓練することができるる。

In [34]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [ ]:
model.fit(train_data, epochs=5)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

In [ ]:
predictions = model.predict(test_data)

# 結果のいくつかを表示
for prediction, cube in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted Cube: {:.4%}".format(prediction[0]),
        " | Actual outcome: ",
        ("CUBE" if bool(cube) else "Capsule"))